# Trabalho prático II - Classificação

O objetivo deste trabalho é praticar os conceitos de aprendizado supervisionado que vimos em sala.

A sua tarefa será treinar um classificador para um conjunto de dados misterioso (se eu falasses qual é o objetivo do modelo, você encontraria soluções na Internet).

Baixe o arquivo [t2.tar.gz](https://drive.google.com/file/d/125plHKUzFGxHjjCiVJcTQG2bPG_zgDNV/view?usp=sharing). Descoprima este arquivo para encontrar outros quatro. Os arquivos `train_X.csv` e `train_y.csv` possuem os dados que você deve usar para treinar o modelo. O arquivo `test_X.csv` possui os objetos para os quais você deve encontrar as classes (testar o modelo). Por fim, o arquivo `test_example_y.csv` é um exemplo de como sua solução final deve ser organizada.

Cada linha dos arquivos `train_X.csv` e `test_X.csv` tem 15 campos descrevendo um objeto misterioso. O campo `id` representa o identificador do objeto, sendo que este campo não deve ser considerado em seu modelo. Os atributos a serem usados no modelo são os 14 campos rotulados de de `a` até `n`. Desses atributos:
- `b`, `d`, `f`, `g`, `h`, `i`, `j` e `n` são categóricos; e
- `a`, `c`, `e`, `k`, `l` e `m` são numéricos.

Cada linha do arquivo `train_y.csv` possui dois campos. O primeiro é o identificador de um objeto do arquivo `train_X.csv` e o segundo é a classe do respectivo objeto (0 ou 1).

Seu objetivo é encontrar as classes dos objetos do arquivo `test_X.csv` e mostrar como chegou em sua solução! Os dados do arquivo de teste foram obtido a partir de uma amostra aleatório do todo. Ou seja, um modelo bem treinado, e sem _overfitting_, em `train_X.csv` e `train_y.csv` se sairá bem em `test_X.csv`.

**Data de entrega:** dia 4 de julho de 2018.

**Grupo:** de até 3 pessoas, mas duas pessoas do mesmo grupo no trabalho 1 não podem pertencer ao mesmo grupo nesse trabalho.

**Valor:** 20% da nota do semestre.

Os três seguintes pontos descrevem o que obrigatoriamente deve ser entregue, com seu respectivo valor.

1 - **[10 pontos]** Este notebook com todo seu código e resultados (números, tabelas e gráficos). Você pode usar qualquer um dos métodos que estudamos ou alguma de suas variações próximas. Se estiver na dúvida se pode usar um método, basta perguntas no Piazza. Comentários e justificativas no notebook não serão considerados para sua nota.
O notebook deve ser enviado para o email do professor.

2 - **[8 pontos]** Um relatório digitado contendo: capa, introdução, metodologia, resultados, conclusão e referências. O relatório deve ter no máximo 10 páginas, com coluna simples, fonte 11, espaçamento 1.5 e margens de 2cm. A seção de metodologia deve conter uma descrição detalhada dos passos seguidos (não incluir código no relatório). A seção de resultados deve conter obrigatoriamente: uma caracterização descritiva dos dados, matriz de confusão das predições, _precision_, _recall_, _F1 score_ e acurácia. Todas as métricas de predição devem ser calculadas a partir dos arquivos de treinamento por meio de validação cruzada.
O relatório deve ser enviado para o email do professor.

3 - **[2 pontos + equivalente a lista extra pela classificação]** A sua predição final do arquivo `test_X.csv` deve ser enviada para o professor por email. O formato deve ser o mesmo do arquivo `train_y.csv`, assim como exemplificado em `test_example_y.csv` (mas repare que as classes desse último arquivo foram gerados de forma aleatória). Em outras palavras, o arquivo a ser entregue deve ter dois campos. O campo `id` é o identificador do objeto em `test_X.csv` e o campo `label` é a classe que seu modelo encontrou para o objeto em questão. A primeira linha do arquivo deve conter os nomes das colunas.
A entrega desse arquivo é obrigatória e vale dois pontos. Além disso, o trabalho com maior _F1 score_ ganhará o equivalente a 100% de uma lista extra. O trabalho com o pior _F1 score_ não ganhará nota extra alguma. Os demais trabalhos terão nota proporcional.
O professor se reserva o direito de anular esse quesito (nota extra) se houver indícios de má conduta durante a competição.

**Kaggle:** Estou tentando criar uma competição para esse trabalho na plataforma _Kaggle_. Se eu conseguir, compartilho o _link_ com você no _Piazza_.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import re
import os
from sklearn import metrics
from sklearn import linear_model
from sklearn import tree
from sklearn import datasets
from sklearn.naive_bayes import BernoulliNB
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
# Lendo arquivos
matriz=[]
matrizTeste=[]
resultados=[]
resultadosTeste=[]
header=[]
headerTeste=[]


# Leitura dos arquivos
with open("train_X.csv", "r") as arquivo:
    spamreader = csv.reader(arquivo, delimiter=',', quotechar='|')
    for index, row in enumerate(spamreader):
        if index!=0:
            matriz.append(row)
        else:
            header = row

with open("test_X.csv", "r") as arquivo:
    spamreader = csv.reader(arquivo, delimiter=',', quotechar='|')
    for index, row in enumerate(spamreader):
        if index!=0:
            matrizTeste.append(row)
        else:
            headerTeste = row            
            
with open("train_y.csv","r") as arquivo:
    spamreader = csv.reader(arquivo, delimiter=',', quotechar='|')
    for index, row in enumerate(spamreader):
        if index!=0:
            resultados.append(row)
        else:
            headerresultados = row                       
    

In [3]:
# Gerando os DataFrames e binarizando as colunas
df = pd.DataFrame(matriz, columns=header)
dfResultados = pd.DataFrame(resultados, columns=headerresultados)
dfTeste = pd.DataFrame(matrizTeste, columns=headerTeste)

tam = len(df)
dfJoin = pd.concat([df, dfTeste])
#transforma os atributos categoricos em colunas binarias
dfAux = pd.get_dummies(dfJoin[['b','d', 'f', 'g', 'h', 'i', 'j','n']])
df = df.drop(['b','d', 'f', 'g', 'h', 'i', 'j','n'], 1)
dfTeste = dfTeste.drop(['b','d', 'f', 'g', 'h', 'i', 'j','n'], 1)

df = df.join(dfAux.iloc[:tam])
dfTeste = dfTeste.join(dfAux.iloc[tam:])


In [4]:
# Fazendo as normalizações
from sklearn import preprocessing
I = np.array(range(len(dfResultados)))
np.random.shuffle(I)

trainSize = int(0.85 * len(I))

#separa conjunto de teste e conjunto de treino
train, test = I[:trainSize], I[trainSize:]
XTrain = preprocessing.normalize(df.drop(['id'],1).values[train])
XTest = preprocessing.normalize(df.drop(['id'],1).values[test])
YTrain = preprocessing.normalize(dfResultados.drop(['id'],1).values[train])
YTest = preprocessing.normalize(dfResultados.drop(['id'],1).values[test])

TesteX =  preprocessing.normalize(dfTeste.drop(['id'],1))

YTrain = [x[0] for x in YTrain]
YTest = [x[0] for x in YTest]


[0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 1.0,
 1.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0

In [5]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier
k_range = [1, 100]
param_grid = dict(n_estimators = k_range)


#floresta para classificação
model = RandomForestClassifier(max_depth=None, random_state=0)
grid = GridSearchCV(model, param_grid, cv=4, scoring='accuracy', n_jobs=2)
grid.fit(XTrain, YTrain)
print (grid.best_params_)
print (grid.best_score_)

{'n_estimators': 100}
0.842985842985843


In [8]:
#testando o conjunto de teste
vector = grid.predict(XTest)


In [12]:
vectorResult = vector
vectorResult = np.array(list(map(lambda x: 1. if x >=1 else 0.,vectorResult)))
vectorResult

final=[]
falsoPositivo=0
falsoNegativo=0
verdadeiroPositivo=0
verdadeiroNegativo=0
total=0

#matriz de confusao
for index,x in enumerate(vectorResult):
    final.append(x==YTest[index])
    if(x==YTest[index]):
        total+=1
    if(x==1 and YTest[index]==1):
        verdadeiroPositivo+=1
    if(x==1 and YTest[index]==0):
        falsoPositivo+=1
    if(x==0 and YTest[index]==1):
        falsoNegativo+=1
    if(x==0 and YTest[index]==0):
        verdadeiroNegativo+=1
    
#valores
print("acuracia: ", len(list(filter(lambda w: w, final))) / (1.0 * len(final)))
print("Precision: ", verdadeiroPositivo/(verdadeiroPositivo+falsoPositivo))
print("Recall: ", verdadeiroPositivo/(verdadeiroPositivo+falsoNegativo))
print("Relação de quantos verdadeiros positivos em relação a quantidade total: " ,verdadeiroNegativo/total)
print("F1: ", 2*((verdadeiroPositivo/(verdadeiroPositivo+falsoPositivo))*(verdadeiroPositivo/(verdadeiroPositivo+falsoNegativo)))/((verdadeiroPositivo/(verdadeiroPositivo+falsoPositivo))+(verdadeiroPositivo/(verdadeiroPositivo+falsoNegativo))))

acuracia:  0.8382499171362281
Precision:  0.707680250783699
Recall:  0.599601593625498
Relação de quantos verdadeiros positivos em relação a quantidade total:  0.8214709371293001
F1:  0.6491732566498922


In [38]:


vectorTeste = grid.predict(TesteX)

writer = csv.writer(open("./resultado.csv", "w"))



writer.writerow(['id','label']) 
for index,x in enumerate(vectorTeste):
    writer.writerow([str(dfTeste.iloc[index]['id']),str(int(x))])  

